# 📚 初始化新的 Questbook（标准结构）

基于280版本的标准结构创建questbook。

In [3]:
import json
import gzip
from pathlib import Path
from datetime import datetime

print("✓ 模块导入成功")

✓ 模块导入成功


## ⚙️ 配置

In [4]:
# 版本配置
VERSION_CODE = "tutorials"
VERSION_NAME = "tutorials 0.01"

# 初始任务线
QUESTLINE = {
    "name": "WelcomeQuests",
    "title_zh": "欢迎任务",
    "title_en": "Welcome Quests"
}

# 初始任务
QUESTS = [
    {
        "name": "Welcome",
        "title_zh": "§6§l欢迎！§r",
        "title_en": "§6§lWelcome!§r",
        "desc_zh": "§e欢迎来到你的任务书！§r",
        "desc_en": "§eWelcome to your questbook!§r",
        "x": 200, "y": 150, "size": 60,
        "is_main": 1, "prereqs": []
    },
    {
        "name": "GettingStarted",
        "title_zh": "§a开始吧",
        "title_en": "§aGetting Started",
        "desc_zh": "§a开始你的旅程§r",
        "desc_en": "§aStart your journey§r",
        "x": 350, "y": 150, "size": 50,
        "is_main": 1, "prereqs": ["Welcome"]
    }
]

print(f"✓ {VERSION_CODE} - {len(QUESTS)} 个初始任务")

✓ tutorials - 2 个初始任务


## 📁 创建目录

In [5]:
BASE = Path("..") / "bin" / "version" / VERSION_CODE
ICONS = BASE / "quests_icons"
QUEST_ICON = ICONS / "QuestIcon"
LINE_ICON = ICONS / "QuestLineIcon"

for d in [BASE, ICONS, QUEST_ICON, LINE_ICON]:
    d.mkdir(parents=True, exist_ok=True)
    print(f"✓ {d.relative_to(Path('..'))}")

✓ bin\version\tutorials
✓ bin\version\tutorials\quests_icons
✓ bin\version\tutorials\quests_icons\QuestIcon
✓ bin\version\tutorials\quests_icons\QuestLineIcon


## 📝 生成任务数据

In [6]:
def gen_data(lang):
    data = {QUESTLINE["name"]: {"data": [], "links": []}}
    for q in QUESTS:
        qid = f"{q['name'].lower()}_{int(datetime.now().timestamp())}"
        q['qid'] = qid
        data[QUESTLINE["name"]]["data"].append({
            "name": q["name"], "symbolSize": q["size"], "symbol": "",
            "x": q["x"], "y": q["y"],
            "data": q[f"desc_{lang}"], "quest_id": qid,
            "title": q[f"title_{lang}"], "is_main": q["is_main"],
            "tooltip": {"show": True}
        })
    for q in QUESTS:
        for p in q["prereqs"]:
            data[QUESTLINE["name"]]["links"].append({
                "source": p, "target": q["name"],
                "symbol": ["none", "arrow"],
                "lineStyle": {
                    "color": "#00c800" if q["is_main"] else "#666666",
                    "width": 4 if q["is_main"] else 2
                },
                "symbolSize": [0, 10]
            })
    return data

with open(BASE / "quest_json.json", 'w', encoding='utf-8') as f:
    json.dump(gen_data('zh'), f, ensure_ascii=False, indent=2)
print("✓ quest_json.json")

with open(BASE / "quest_json_en.json", 'w', encoding='utf-8') as f:
    json.dump(gen_data('en'), f, ensure_ascii=False, indent=2)
print("✓ quest_json_en.json")

✓ quest_json.json
✓ quest_json_en.json


## 📋 任务线列表

In [7]:
lines = [{
    "title": QUESTLINE["title_en"],
    "title_zh": QUESTLINE["title_zh"],
    "quest": QUESTLINE["name"]
}]

with open(BASE / "quest_line.json", 'w', encoding='utf-8') as f:
    json.dump(lines, f, ensure_ascii=False, indent=2)
print("✓ quest_line.json")

✓ quest_line.json


## 🎨 创建图标文件（标准结构）

In [8]:
DUMMY = "iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAAAADUlEQVR42mNk+M9QDwADhgGAWjR9awAAAABJRU5ErkJggg=="

def save_gtbl(path, data):
    """保存JSON和GTBL"""
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path.with_suffix('.json'), 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    with gzip.open(path, 'wb') as f:
        f.write(json.dumps(data, ensure_ascii=False).encode('utf-8'))
    print(f"  ✓ {path.relative_to(Path('..'))}")

# 1. 任务线图标：QuestLineIcon/WelcomeQuests.gtbl
save_gtbl(LINE_ICON / f"{QUESTLINE['name']}.gtbl", {QUESTLINE['name']: DUMMY})

# 2. 任务图标：QuestIcon/WelcomeQuests.gtbl（包含所有任务）
quest_icons = {q['qid']: DUMMY for q in QUESTS}
save_gtbl(QUEST_ICON / f"{QUESTLINE['name']}.gtbl", quest_icons)

print("\n✓ 图标文件创建完成（占位符）")

  ✓ bin\version\tutorials\quests_icons\QuestLineIcon\WelcomeQuests.gtbl
  ✓ bin\version\tutorials\quests_icons\QuestIcon\WelcomeQuests.gtbl

✓ 图标文件创建完成（占位符）


## 🔗 图集配置

In [9]:
atlas = {
    "QuestLineIcon": [QUESTLINE['name']],
    f"QuestIcon/{QUESTLINE['name']}": [q['qid'] for q in QUESTS]
}

with open(BASE / "quests_icons.json", 'w', encoding='utf-8') as f:
    json.dump(atlas, f, ensure_ascii=False, indent=2)
print("✓ quests_icons.json")

✓ quests_icons.json


## ⚙️ 更新项目配置

In [10]:
import re
cfg = Path("..") / "src" / "ProjectConfig.ts"

if cfg.exists():
    txt = cfg.read_text(encoding='utf-8')
    cfg.with_suffix('.ts.backup').write_text(txt, encoding='utf-8')
    
    txt = re.sub(
        r'(versionList = \[)([^\]]*)(\])',
        lambda m: f'{m[1]}{m[2].strip()}, "{VERSION_NAME}"{m[3]}',
        txt
    )
    txt = re.sub(
        r'(resList = \[)([^\]]*)(\])',
        lambda m: f'{m[1]}{m[2].strip()}, "{VERSION_CODE}"{m[3]}',
        txt
    )
    
    cfg.write_text(txt, encoding='utf-8')
    print("✓ ProjectConfig.ts 已更新")
else:
    print("⚠️ 需手动更新 ProjectConfig.ts")

✓ ProjectConfig.ts 已更新


## ✅ 完成

In [11]:
print("="*60)
print("✨ Questbook 初始化完成！")
print("="*60)
print(f"版本: {VERSION_NAME} ({VERSION_CODE})")
print(f"任务: {len(QUESTS)} 个")
print(f"\n📁 标准文件结构:")
print(f"  quests_icons/QuestLineIcon/{QUESTLINE['name']}.gtbl")
print(f"  quests_icons/QuestIcon/{QUESTLINE['name']}.gtbl")
print(f"  quest_json.json, quest_json_en.json")
print(f"  quest_line.json, quests_icons.json")
print(f"\n🔧 下一步:")
print(f"  1. Commit 并 push 到 GitHub")
print(f"  2. 等待自动构建和部署")
print(f"  3. 或本地测试: npm run build && npm run start")
print("="*60)

✨ Questbook 初始化完成！
版本: tutorials 0.01 (tutorials)
任务: 2 个

📁 标准文件结构:
  quests_icons/QuestLineIcon/WelcomeQuests.gtbl
  quests_icons/QuestIcon/WelcomeQuests.gtbl
  quest_json.json, quest_json_en.json
  quest_line.json, quests_icons.json

🔧 下一步:
  1. Commit 并 push 到 GitHub
  2. 等待自动构建和部署
  3. 或本地测试: npm run build && npm run start
